# **Fine-tune a RoBERTa model trained on MLM for Text Generation**

## *Warm-starting RoBERTa for Product Names Generation*

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


## Loading the libraries

In [ ]:
#Installation
!pip install datasets==1.0.2
!pip install transformers
!rm seq2seq_trainer.py
#!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/seq2seq/seq2seq_trainer.py
!pip install rouge_score

import datasets
import transformers
import pandas as pd
from datasets import Dataset

#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

import os

     |████████████████████████████████| 1.8MB 8.2MB/s 
     |████████████████████████████████| 245kB 32.1MB/s 
     |████████████████████████████████| 1.9MB 8.0MB/s 
     |████████████████████████████████| 890kB 36.5MB/s 
     |████████████████████████████████| 3.2MB 45.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ef3d37917f9b7deeb6117d655241b20d6a15f8d543c00bb68683d3ccfa98732d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
rm: cannot remove 'seq2seq_trainer.py': No such file or directory
--2021-02-27 11:41:15--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/legacy/seq2seq/seq2seq_trainer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.

# Loading the dataset

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Define parameters for data location and model folders

In [ ]:
#Set the path to the data folder, datafile and output folder and files
root_folder = '/content/drive/My Drive/'
data_folder = os.path.abspath(os.path.join(root_folder, 'datasets/text_gen_product_names'))
model_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names/RoBERTa-FT-MLM'))
output_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names'))
pretrainedmodel_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names/TokRoBERTa'))
decoder_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names/DecRoBERTaML'))

test_filename='cl_test_descriptions.csv'
datafile= 'product_names_desc_cl_train.csv'
outputfile = 'submission.csv'

datafile_path = os.path.abspath(os.path.join(data_folder,datafile))
testfile_path = os.path.abspath(os.path.join(data_folder,test_filename))
outputfile_path = os.path.abspath(os.path.join(output_folder,outputfile))

Load the datafile with the product descriptions and names:

In [ ]:
# Load the dataset: sentence in english, sentence in spanish 
df=pd.read_csv(datafile_path, header=0, usecols=[0,1])
print('Num Examples: ',len(df))
print('Null Values\n', df.isna().sum())

Num Examples:  31593
Null Values
 name           44
description     1
dtype: int64


Remove rows with null values:

In [ ]:
df.dropna(inplace=True)
print('Num Examples: ',len(df))

Num Examples:  31548


## Split the data into train and validation dataset

In [ ]:
# Creation of Dataset and Dataloader
# Defining the train size. So 90% of the data will be used for training and the rest will be used for validation. 
train_size = 0.9
train_dataset=df.sample(frac=train_size,random_state = 42)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
print('Length Train dataset: ', len(train_dataset))
print('Length Val dataset: ', len(val_dataset))

Length Train dataset:  28393
Length Val dataset:  3155


In [ ]:
# To limit the training and validation dataset, for testing
max_train=28393
max_val=3155

train_data=Dataset.from_pandas(train_dataset[:max_train])
val_data=Dataset.from_pandas(val_dataset[:max_val])

# Setting the model and training parameters

In [ ]:
TRAIN_BATCH_SIZE = 32   #32    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 4    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 15        # number of epochs to train (default: 10)
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
SEED = 42               # random seed (default: 42)
MAX_LEN = 100
SUMMARY_LEN = 7

In [ ]:
train_data

Dataset(features: {'name': Value(dtype='string', id=None), 'description': Value(dtype='string', id=None)}, num_rows: 28393)

## Load the trained tokenizer on our especific language 

In [ ]:
# Loading the RoBERTa Tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained(pretrainedmodel_folder)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

## Prepare and create the Dataset

In [ ]:
batch_size=TRAIN_BATCH_SIZE  # change to 16 for full training
encoder_max_length=MAX_LEN
decoder_max_length=SUMMARY_LEN

def process_data_to_model_inputs(batch):
  inputs = tokenizer(batch["description"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["name"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["description", "name"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["description", "name"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
#dataset = dataset.shuffle(seed=42, buffer_size=10, reshuffle_each_iteration=True)


# Create the Transformer Encoder-Decoder model

In [ ]:
# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained(pretrainedmodel_folder, decoder_folder, tie_encoder_decoder=False)

Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/My Drive/Projects/SpainAI NLP/TokRoBERTa and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at /content/drive/My Drive/Projects/SpainAI NLP/DecRoBERTaML and are newly initialized: ['roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.

In [ ]:
print('Vocab Size: ',roberta_shared.config.encoder.vocab_size)

Vocab Size:  8192


In [ ]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id
#roberta_shared.config.eos_token_id = tokenizer.eos_token_id
# Testing the next assigment
#roberta_shared.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = SUMMARY_LEN
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 1
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.repetition_penalty = 3.0
# Must change for testing?
roberta_shared.config.num_beams = 10
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

Create the Seq2SeqTrainingArguments class to include some required parameters:

In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

## Define the Rouge metric for model evaluation

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Set the training arguments:

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=model_folder,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    #evaluate_during_training=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=1024,  
    save_steps=2048, 
    #eval_steps=256, 
    warmup_steps=1024,  
    #max_steps=1500, # delete for full training
    num_train_epochs = 10, #TRAIN_EPOCHS,
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=True, 
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

The `config.pad_token_id` is `None`. Using `config.eos_token_id` = 2 for padding..


Now, we start training the model:

In [ ]:
# Fine-tune the model, training and evaluating on the tran dataset
trainer.train()

/usr/local/lib/python3.7/dist-packages/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Runtime,Samples Per Second
1,No log,2.730064,0.135200,0.138400,0.132600,86.068500,36.657000
2,3.454800,2.336565,0.213900,0.221400,0.211100,88.289700,35.735000
3,2.381900,2.124129,0.265100,0.274900,0.262700,88.231800,35.758000
4,1.925600,2.003713,0.291400,0.276000,0.276100,87.047200,36.245000
5,1.592000,1.915785,0.299800,0.313300,0.298600,89.034100,35.436000
6,1.332000,1.881782,0.322800,0.328500,0.317800,87.646800,35.997000
7,1.120100,1.864490,0.342800,0.332500,0.329400,87.338900,36.124000
8,1.120100,1.848708,0.344600,0.351500,0.340300,88.478400,35.658000
9,0.943200,1.854323,0.344400,0.346900,0.338400,87.868400,35.906000
10,0.816400,1.856113,0.346100,0.347100,0.338900,88.329500,35.719000


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

TrainOutput(global_step=8880, training_loss=1.6225291140444644, metrics={'train_runtime': 2978.7447, 'train_samples_per_second': 2.981, 'total_flos': 20744873844541440, 'epoch': 10.0})

Save the model just trained:

In [ ]:
trainer.save_model(model_folder)

# Evaluate the model on the test dataset

In [ ]:
# Load the dataset: sentence in english, sentence in spanish 
df=pd.read_csv(testfile_path, header=0)
print('Num Examples: ',len(df))
print('Null Values\n', df.isna().sum())
print(df.head(5))

Num Examples:  1441
Null Values
 description    0
dtype: int64
                                         description
0  knit midi dress with vneckline straps matching...
1  loosefitting dress with round neckline long sl...
2  nautical with peak.this item must returned wit...
3  nautical with peak . adjustable inner strap de...
4  nautical with side button detail.this item mus...


In [ ]:
test_data=Dataset.from_pandas(df)
print(test_data)

Dataset(features: {'description': Value(dtype='string', id=None)}, num_rows: 1441)


In [ ]:
checkpoint_path = os.path.abspath(os.path.join(model_folder,'checkpoint-3072'))
print(checkpoint_path)

Define the Tokenizer and load the fine-tuned model:

In [ ]:
#Load the Tokenizer and the fine-tuned model
tokenizer = RobertaTokenizerFast.from_pretrained(pretrainedmodel_folder)
model = EncoderDecoderModel.from_pretrained(model_folder)

model.to("cuda")


The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(8192, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

Define various method to generate text using beam search and random sampling:

In [ ]:
# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["description"], padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    #outputs = roberta_shared.generate(input_ids, attention_mask=attention_mask)
    outputs = roberta_shared.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch


In [ ]:
# Generate a text using beams search
def generate_summary_beam_search(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["description"], padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = roberta_shared.generate(input_ids, attention_mask=attention_mask,
                                  num_beams=15,
                                  repetition_penalty=3.0, 
                                  length_penalty=2.0, 
                                  num_return_sequences = 1
    )

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

# Generate a text using beams search
def generate_summary_topk(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["description"], padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = roberta_shared.generate(input_ids, attention_mask=attention_mask,
                                  repetition_penalty=3.0, 
                                  length_penalty=2.0, 
                                  num_return_sequences = 1,
                                  do_sample=True,
                                  top_k=50, 
                                  top_p=0.95,

    )

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch


Make predictions for the test dataset:

In [ ]:
#test_data = test_data.select(range(5000))
batch_size = TRAIN_BATCH_SIZE

#results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["description"])
results = test_data.map(generate_summary_beam_search, batched=True, batch_size=batch_size, remove_columns=["description"])
#results = test_data.map(generate_summary_topk, batched=True, batch_size=batch_size, remove_columns=["description"])

pred_str = results["pred"]
#label_str = results["Summary"]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [ ]:
len(pred_str)

1441

When more than one output are generated we need to join them on a single list 

In [ ]:
import numpy as np

preds=np.reshape(pred_str, (-1, 3))
print('Predictions Shape: ',preds.shape)
predictions = [','.join(p) for p in preds]
print('Num predictions: ', len(predictions),predictions)

Predictions Shape:  (1441, 3)
Num predictions:  1441 ['lace midi dress trf,lacetrimmed camisole dress,lacetrimmed dress tr', 'pleated dress trf,floral print dress,poplin dress', 'corduroy nautical cap,check nautical cap,checked nautical cap', 'corduroy nautical cap,faux shearling nautical cap,check nautical cap', 'check nautical cap,checked nautical cap,corduroy nautical cap', 'cropped tiedye tshirt trf,fadedeffect tshirt print trf,fadedeffect tshirt slogan trf', 'faux suede coat,coat faux suede,doublefaced faux suede coat', 'full sleeve tshirt,fringed tshirt,balloon sleeve tshirt', 'stretch top,stretch top trf,stretch top straps', 'stretch top trf,stretch top straps trf,stretch top', 'stretch top straps trf,stretch top,stretch top trf', 'stretch top straps trf,stretch top trf,stretch top straps', 'stretch top straps trf,stretch top trf,cropped vest top trf', 'stretch top straps trf,stretch top trf,stretch top straps', 'stretch top,stretch top trf,stretch top straps trf', 'mini dress p

In [ ]:
print(predictions)

['lace midi dress trf,lacetrimmed camisole dress,lacetrimmed dress tr', 'pleated dress trf,floral print dress,poplin dress', 'corduroy nautical cap,check nautical cap,checked nautical cap', 'corduroy nautical cap,faux shearling nautical cap,check nautical cap', 'check nautical cap,checked nautical cap,corduroy nautical cap', 'cropped tiedye tshirt trf,fadedeffect tshirt print trf,fadedeffect tshirt slogan trf', 'faux suede coat,coat faux suede,doublefaced faux suede coat', 'full sleeve tshirt,fringed tshirt,balloon sleeve tshirt', 'stretch top,stretch top trf,stretch top straps', 'stretch top trf,stretch top straps trf,stretch top', 'stretch top straps trf,stretch top,stretch top trf', 'stretch top straps trf,stretch top trf,stretch top straps', 'stretch top straps trf,stretch top trf,cropped vest top trf', 'stretch top straps trf,stretch top trf,stretch top straps', 'stretch top,stretch top trf,stretch top straps trf', 'mini dress pockets,dress contrast pockets,ruffled dress', 'stripe

Save the predictions to a file:

In [ ]:
outputfile_path

'/content/drive/My Drive/Projects/SpainAI NLP/submission.csv'

In [ ]:
final_df = pd.DataFrame({'name':pred_str})
final_df.to_csv(outputfile_path, index=False)
print('Output Files generated for review')

Output Files generated for review


In [ ]:
#1000
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE F SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

ROUGE 1 SCORE:  Score(precision=0.32608695652173914, recall=0.22801383399209485, fmeasure=0.25677004155265026)
ROUGE 2 SCORE:  Score(precision=0.17391304347826086, recall=0.15217391304347827, fmeasure=0.15942028985507245)
ROUGE F SCORE:  Score(precision=0.32608695652173914, recall=0.22801383399209485, fmeasure=0.25677004155265026)
